In [1]:
# %load ../first.cell
%load_ext autoreload
%autoreload 2

In [ ]:
model_fp32 = "./vgg16-12.onnx"

import torch
import torchvision.datasets as datasets
from torch.utils.data import Dataloader
val_dataset = datasets.CIFAR10("./data",
                             train=False,
                             download=True)
val_dataloader = Dataloader(
                     val_dataset,
                     batch_size=32, shuffle=False,
                     ping_memory=True)

In [ ]:
import deepcrunch

In [ ]:
quantized_model = deepcrunch.quantize(model_fp32, output_path="./vgg16_quantize_nc.onnx", val_dataset=val_dataset)

In [ ]:
# If calibration or accuracy aware needed
eval_func = # Some function that takes in a model and returns accuracy or metrics. Usually a validation function.

quantized_model = deepcrunch.quantize(model_fp32, output_path="./vgg16_quantize_nc.onnx", val_dataset=val_dataset, eval_func=eval_func)

In [ ]:
original_model_size = deepcrunch.performance.size_in_mb(
    "./vgg16-12.onnx", human_readable=True
)

quantized_model_size = deepcrunch.performance.size_in_mb(
    "./vgg16_quantize_nc.onnx", human_readable=True
)

print(f"Original model size: {original_model_size}")
print(f"Quantized model size: {quantized_model_size}")

Original model size: 527.80 MB
Quantized model size: 132.01 MB


In [ ]:
import torch
import onnx
import onnxruntime as ort

print(torch.__version__)

device_id = 0
print(ort.get_device()) 

1.11.0
GPU


In [ ]:
providers = [(
    "CUDAExecutionProvider", 
    { 
        "device_id": device_id,
        "cudnn_conv_algo_search": "HEURISTIC", 
        "do_copy_in_default_stream": True
    }
)]

opts = ort.SessionOptions()
opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
opts.enable_cpu_mem_arena = False
opts.enable_mem_pattern = False
opts.enable_mem_reuse = False

opts.intra_op_num_threads = 1 
opts.inter_op_num_threads = 1

sess = ort.InferenceSession(
    "vgg16_quantize_nc.onnx", 
    sess_options=opts, 
    providers=providers)

In [ ]:
from deepcrunch.performance.model_latency import do_inference

do_inference("vgg16-12.onnx", "VGG16-12", input_dim=(1, 3, 224, 224), providers=providers)
do_inference("vgg16_float16.onnx", "Float16", input_dim=(1, 3, 224, 224), providers=providers)

2023-08-02 11:56:30.592380154 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87018, index: 18, mask: {19, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-08-02 11:56:30.575542837 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87016, index: 16, mask: {17, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-08-02 11:56:30.575538211 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87011, index: 11, mask: {12, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-08-02 11:56:30.588366443 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87014, index: 14, mask: {15, }, error code: 22 error msg: Invalid argument. Specify the n

100%|██████████| 100/100 [00:00<00:00, 209.40it/s, time=3.22ms]
2023-08-02 11:56:32.091381691 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87036, index: 12, mask: {13, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.


VGG16-12 Average Time: 4.805


2023-08-02 11:56:32.091381851 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87035, index: 11, mask: {12, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-08-02 11:56:32.091389060 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87037, index: 13, mask: {14, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-08-02 11:56:32.101366702 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87042, index: 18, mask: {19, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-08-02 11:56:32.103362427 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 87041, index: 17, mask: {18, }, error code: 22 error msg: Invalid argument. Specify the n

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(float)) , expected: (tensor(float16))